## DNN Serving System

In [ ]:
import torch
import os
import sys
base_dir = os.environ.get('TORCHSIM_DIR', default='/workspace/PyTorchSim')
sys.path.append(base_dir)

### Scheduler

In [ ]:
import torch
from torchvision.models import resnet18
from Scheduler.scheduler import Scheduler, SchedulerDNNModel, Request
from PyTorchSimFrontend import extension_config

scheduler = Scheduler(num_request_queue=1, engine_select=Scheduler.FIFO_ENGINE, togsim_config=extension_config.TOGSIM_CONFIG)
device = scheduler.execution_engine.module.custom_device()

model = resnet18().eval()
input = torch.randn(1, 3, 224, 224).to(device=device)
opt_fn = torch.compile(dynamic=False)(model.to(device, memory_format=torch.channels_last))

SchedulerDNNModel.register_model("resnet18", opt_fn)
request = Request("resnet18", [input], [], request_queue_idx=0)
scheduler.add_request(request, request_time=0)

# Run scheduler
while not scheduler.is_finished():
    with torch.no_grad():
        scheduler.schedule()

print("ResNet18 Simulation Done")

### Load Generator

In [ ]:
import os
import torch
from torchvision.models import resnet18

from Scheduler.scheduler import Scheduler, SchedulerDNNModel, Request, poisson_request_generator
TORCHSIM_DIR = os.environ.get('TORCHSIM_DIR', default='/workspace/PyTorchSim')

lambda_requests = 10
max_time = 30

target_model1 = resnet18().eval()

# Init scheduler
scheduler = Scheduler(num_request_queue=1, max_batch=32, engine_select=Scheduler.FIFO_ENGINE, togsim_config=extension_config.TOGSIM_CONFIG)
# Register compiled model
opt_model1 = torch.compile(target_model1.to(device=scheduler.execution_engine.module.custom_device(), memory_format=torch.channels_last), dynamic=False)
SchedulerDNNModel.register_model("resnet18", opt_model1)

# Generate time stamp
for request_time in poisson_request_generator(lambda_requests, max_time):
    # Init input data
    model_input1 = torch.randn(1, 3, 224, 224)

    # Init request
    new_request1 = Request("resnet18", [model_input1], [], request_queue_idx=0)

    # Add request to scheduler
    print("[Reqest] Resnet18 request time: ", request_time, flush=True)
    scheduler.add_request(new_request1, request_time=request_time)

# Run scheduler
while not scheduler.is_finished():
    scheduler.schedule()